In [11]:
# Loading data
from tools.data_loader import load_dat
lines = load_dat("../btc_data/bh.dat")
print(lines[:5])

[('0', '000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f', '1231006505', '1'), ('1', '00000000839a8e6886ab5951d76f411475428afc90947ee320161bbf18eb6048', '1231469665', '1'), ('2', '000000006a625f06636b8bb6ac7b960a8d03705d1ace08b1a19da3fdcc99ddbd', '1231469744', '1'), ('3', '0000000082b5015589a3fdf2d4baff403e6f0be035a5d9742c1cae6295464449', '1231470173', '1'), ('4', '000000004ebadb55ee9096c9a2f8880e09da59c0d68b1c228da88e48844a1485', '1231470988', '1')]
